In [1]:
import os
from pathlib import Path
from PIL import Image
import cv2
import numpy as np
import json
from shapely.geometry import Polygon
import image_slicer

# DeepLab code:
# taken from https://gluon-cv.mxnet.io/build/examples_segmentation/demo_deeplab.html
# dataset description https://groups.csail.mit.edu/vision/datasets/ADE20K/, https://github.com/dmlc/gluon-cv/blob/master/gluoncv/data/ade20k/segmentation.py
# deeplab code https://github.com/dmlc/gluon-cv/blob/master/gluoncv/model_zoo/deeplabv3.py

import mxnet as mx
from mxnet import image
from mxnet.gluon.data.vision import transforms
import gluoncv
from gluoncv.data.transforms.presets.segmentation import test_transform
# using cpu
ctx = mx.cpu(0)

# OCR code
import pytesseract
from pytesseract import Output


# If you don't have tesseract executable in your PATH, include the following:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe' # Path('C:/Program Files\ Tesseract-OCR\ tesseract').as_posix()


# Scene recognition imports
from Keras_VGG16_places365.vgg16_places_365 import VGG16_Places365
from cv2 import resize


import evaluation_utils as eval_u
import deeplab_utils as dl_u

Using TensorFlow backend.


In [2]:
#image_list_deeplab, list_image_names_deeplab, list_ratio = eval_u.prepare_dataset('../small_10_test2017/', 'deeplab', True)
image_list_deeplab, list_image_names_deeplab, list_ratio = eval_u.prepare_dataset('../1_test2017/', 'deeplab', True)

In [3]:
model_deeplab = eval_u.load_model('deeplab')
output_pred_deeplab = eval_u.get_predictions(image_list_deeplab, 'deeplab', model_deeplab)

Currently treating image number  1
predicting
calculating labels
0.0
calculating probabilities.
37.20501136779785


In [4]:
#print(output_pred_deeplab[0].shape)

In [5]:
#outp = mx.nd.squeeze(mx.nd.argmax(output_pred_deeplab[0], 1)).asnumpy()
print(output_pred_deeplab[0][1].shape)
print(type(output_pred_deeplab[0][1]))
#print(np.unique(output_pred_deeplab[0][1]))

(512, 682)
<class 'mxnet.ndarray.ndarray.NDArray'>


In [6]:
output = dl_u.deeplab_pred_to_output(output_pred_deeplab[0][1])

TODO: manage resizing of images.
Creating masks.
Number of masks to create:  (8,)
Creating mask for class  0.0
Creating mask for class  12.0
Creating mask for class  22.0
Creating mask for class  37.0
Creating mask for class  43.0
Creating mask for class  45.0
Creating mask for class  69.0
Creating mask for class  121.0
Creating polygons.
--getting contour
--transforming contour to poygon.
--getting contour
--transforming contour to poygon.
--getting contour
--transforming contour to poygon.
--getting contour
--transforming contour to poygon.
--getting contour
--transforming contour to poygon.
--getting contour
--transforming contour to poygon.
--getting contour
--transforming contour to poygon.
--getting contour
--transforming contour to poygon.


In [ ]:
def simple_rule_based_obfuscation(output_deeplab):
    # Later we can save other things like the "sky" to decide to hide it
    true_output = []
    for poly in output_deeplab:
        if (poly[1] == "person, individual, someone, somebody, mortal, soul") or (poly[1] == "bookcase") or (poly[1] == "computer, computing machine, computing device, data processor, ") or (poly[1] == "trade name, brand name, brand, marque") :
            true_output.append(poly)
    return true_output

In [ ]:
print(type(output_pred_deeplab[0]))

In [7]:
print(output)

[([<shapely.geometry.polygon.Polygon object at 0x0000023464656648>, <shapely.geometry.polygon.Polygon object at 0x00000234633B0208>, <shapely.geometry.polygon.Polygon object at 0x00000234636C5D48>, <shapely.geometry.polygon.Polygon object at 0x0000023464656288>, <shapely.geometry.polygon.Polygon object at 0x0000023464656088>, <shapely.geometry.polygon.Polygon object at 0x0000023464656208>, <shapely.geometry.polygon.Polygon object at 0x0000023464656888>, <shapely.geometry.polygon.Polygon object at 0x0000023464656148>, <shapely.geometry.polygon.Polygon object at 0x000002346463FEC8>, <shapely.geometry.polygon.Polygon object at 0x0000023464656688>, <shapely.geometry.polygon.Polygon object at 0x0000023464656708>, <shapely.geometry.polygon.Polygon object at 0x0000023464656748>, <shapely.geometry.polygon.Polygon object at 0x0000023464656AC8>, <shapely.geometry.polygon.Polygon object at 0x00000234646553C8>, <shapely.geometry.polygon.Polygon object at 0x0000023464655288>, <shapely.geometry.poly

In [ ]:
true_output = simple_rule_based_obfuscation(output)

In [ ]:
print(true_output)

In [ ]:
with open(Path('../test2017.json'), 'r') as f:
    ground_truth = json.load(f)
ground_truth = ground_truth['annotations']

In [ ]:
eval_u.evaluate(ground_truth, true_output, evaluation_type, parameter_interval)